Find recoherence time ( the time != 0 ) where the purity of the state is 1 again. 

Purity of state is calculated using the density matrix using the forumula below.
$ p = tr(\rho^2) $

The following source does the following.
- Start with an initial quantum state described the RY operators below for 5 qubits. 
- Evolve the state using coefficiennts and operators that construct the interaction hamiltonian.
- Calculate purity by the formula above for the quantum state. 
- Evolve the state and measure the closeness to one of the purity of the state. 

In [50]:
import json
import pennylane as qml
import pennylane.numpy as np

dev = qml.device("default.qubit", wires=5)

@qml.qnode(dev)
def evolve_state(coeffs, time):
    operators = [
        qml.PauliZ(0) @ qml.PauliZ(1),
        qml.PauliZ(0) @ qml.PauliZ(2),
        qml.PauliZ(0) @ qml.PauliZ(3),
        qml.PauliZ(0) @ qml.PauliZ(4),
    ]

    qml.RY(0.5 * np.pi, wires=0)
    qml.RY(0.4 , wires=1)
    qml.RY(1.2 , wires=2)
    qml.RY(1.8 , wires=3)
    qml.RY(0.6 , wires=4)

    qml.ApproxTimeEvolution(qml.Hamiltonian(coeffs, operators), time, n=1)
    return qml.density_matrix(wires=[0])

def purity(rho):
    return qml.math.purity(rho, [0,1,2,3,4])

def recoherence_time(coeffs):
    start = 0.1
    for t in range(10000):
        t = start + t*0.01
        rho = evolve_state(coeffs, t)
        puriity = purity(rho)
        if np.isclose(puriity, 1, 1e-2):
            return t
    return t
    # Return the recoherence time


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    params = json.loads(test_case_input)
    output = recoherence_time(params)

    return str(output)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.isclose(solution_output, expected_output, rtol=5e-2)


# These are the public test cases
test_cases = [
    ('[5,5,5,5]', '0.314'),
    ('[1.1,1.3,1,2.3]', '15.71')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)
        print (output)
    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[5,5,5,5]'...
0.31
Correct!
Running test case 1 with input '[1.1,1.3,1,2.3]'...
15.68
Correct!
